In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movie_conversations.txt to movie_conversations.txt
Saving movie_lines.txt to movie_lines.txt


In [ ]:
!pip install transformers datasets

In [ ]:
# Step 1: Parse movie_lines.txt
id2line = {}
with open("movie_lines.txt", encoding="ISO-8859-1") as f:  # You can try changing this and see what changes in the response you get :)
    for line in f:
        parts = line.strip().split(" +++$+++ ")
        if len(parts) == 5:
            line_id = parts[0]
            text = parts[4]
            id2line[line_id] = text

# Step 2: Parse movie_conversations.txt into a list of conversations
conversations = []
with open("movie_conversations.txt", encoding="ISO-8859-1") as f:  # Make sure to keet encodings consistent
    for line in f:
        parts = line.strip().split(" +++$+++ ")
        if len(parts) == 4:
            try:
                utterance_ids = eval(parts[3])  # Converts string list to actual list
                conversations.append(utterance_ids)
            except Exception as e:
                print(f"Skipping line due to eval error: {e}")

# Step 3: Build input-output pairs (prompt-response)
pairs = []
for conv in conversations:
    for i in range(len(conv) - 1):
        if conv[i] in id2line and conv[i+1] in id2line:
            input_line = id2line[conv[i]].strip()
            target_line = id2line[conv[i+1]].strip()
            if input_line and target_line:  # skip empty lines
                pairs.append((input_line, target_line))


print(f"Loaded {len(pairs)} dialog pairs.")

Loaded 221282 dialog pairs.


In [ ]:
import random

# Parameters
SAMPLE_SIZE = 5_000      # how many pairs you want, you can change it
RANDOM_SEED = 42          # set this if you need deterministic sampling

# Draw the sample
random.seed(RANDOM_SEED)          # comment this out for a fresh shuffle each run
sample_pairs = random.sample(pairs, SAMPLE_SIZE)

print(f"Sampled {len(sample_pairs)} pairs.")

Sampled 5000 pairs.


In [ ]:
from datasets import Dataset

# Create a Hugging Face Dataset from your list of (input, output) pairs
data = [{"input": q, "output": a} for q, a in sample_pairs]
hf_dataset = Dataset.from_list(data)

print(hf_dataset[0])  # sanity check

{'input': "You didn't come here to destroy Wintermute. You can to save a man you love. A man who isn't even capable of returning that love. Such a waste...", 'output': "My man's coming to get my ass out of here. That's good enough for me."}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.pad_token = tokenizer.eos_token  # Fix the pad token issue


def tokenize(example):
    input_text = "Human: " + example["input"] + "\nBot: "
    output_text = example["output"] + tokenizer.eos_token
    # input_text = example["input"] + tokenizer.eos_token
    # output_text = example["output"] + tokenizer.eos_token
    full_text = input_text + output_text
    tokens = tokenizer(full_text, truncation=True, padding="max_length", max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()  # Causal language modeling
    return tokens

tokenized_dataset = hf_dataset.map(tokenize, batched=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
# Need to import get_last_checkpoint from the trainer_utils module
from transformers.trainer_utils import get_last_checkpoint


# 1. Detect an existing checkpoint (if any)

# output_dir = "./dialogpt-finetuned"
output_dir = "./dialogpt-finetuned-fv"
last_ckpt  = get_last_checkpoint(output_dir) if os.path.isdir(output_dir) else None
if last_ckpt:
    print(f"  Found checkpoint at: {last_ckpt} – resuming from there.")


# 2. Load model (fresh or from checkpoint)

model_name_or_path = last_ckpt or "microsoft/DialoGPT-medium"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

# (Optional but tidy) – make sure pad token is set

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.pad_token = tokenizer.eos_token


training_args = TrainingArguments(
    output_dir           = output_dir,
    per_device_train_batch_size = 3,
    num_train_epochs     = 3,
    dataloader_num_workers = 4,
    learning_rate=7e-5,

    # logging & checkpointing
    logging_strategy     = "steps",
    logging_steps        = 200,
    save_strategy        = "steps",
    save_steps           = 500,
    save_total_limit     = 2,

    # misc
    fp16                 = True,     # comment out if GPU doesn’t support fp16
    report_to            = "none",   # no WandB/HF Hub logging
)


# 4. Trainer
trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = tokenized_dataset,
    tokenizer     = tokenizer,  # keeps pad/eos alignment neat
)


# 5. Train – resume if we have a checkpoint
trainer.train(resume_from_checkpoint=last_ckpt)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipython-input-4195498462.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,1.187100
400,0.800200
600,0.881300
800,0.884200
1000,0.805600
1200,0.831900
1400,0.854100
1600,0.869700
1800,0.738100
2000,0.693900


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Step,Training Loss
200,1.187100
400,0.800200
600,0.881300
800,0.884200
1000,0.805600
1200,0.831900
1400,0.854100
1600,0.869700
1800,0.738100
2000,0.693900


TrainOutput(global_step=5001, training_loss=0.6963792492260673, metrics={'train_runtime': 2628.6995, 'train_samples_per_second': 5.706, 'train_steps_per_second': 1.902, 'total_flos': 3482627604480000.0, 'train_loss': 0.6963792492260673, 'epoch': 3.0})

In [ ]:
# Saving the freshly trained moel and its tokeniser
trainer.save_model("./dialogpt-finetuned-fv/final")
tokenizer.save_pretrained("./dialogpt-finetuned-fv/final")

('./dialogpt-finetuned-fv/final/tokenizer_config.json',
 './dialogpt-finetuned-fv/final/special_tokens_map.json',
 './dialogpt-finetuned-fv/final/chat_template.jinja',
 './dialogpt-finetuned-fv/final/vocab.json',
 './dialogpt-finetuned-fv/final/merges.txt',
 './dialogpt-finetuned-fv/final/added_tokens.json',
 './dialogpt-finetuned-fv/final/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the fine-tuned model
model_path = "./dialogpt-finetuned-fv/final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model     = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()

# Chat prompt
chat_history = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]: break

    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat(chat_history + [new_input_ids], dim=-1) if chat_history else new_input_ids

    output_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9,
    )

    response = tokenizer.decode(output_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Bot: {response}")
    chat_history.append(new_input_ids)


You: hey


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: Hey friendo
You: what you doing
Bot: ... I was helping you with something. I wasn't helping you with anything.
Bot: Well... what?
You: what
Bot: just messing around
You: youre funny
Bot: You're a funny guy.
You: im not a guy
Bot: but you sure are funny.
You: thank you
Bot: you're welcome
You: okay bye
Bot: Byebye. See you.
You: exit
